In [4]:
import sys
import os
import pandas as pd

In [5]:
df = pd.read_parquet("../sourcedata/df_metrics.parquet")
df.sort_values(by=['scene_full_name', 'subject', 'learning_phase'], inplace=True)
df

,subject,learning_phase,scene_full_name,count,cleared,speed,MAD_mean,delta_clr_tot,delta_spd_tot,delta_MAD_tot,level_full_name,scene
3267,im_sub-01,sub-01_epoch=0-step=2000,1-1-0,44,1.000000,65.546218,NaN,-0.909091,-59.380211,NaN,w1l1,0
3847,im_sub-01,sub-01_epoch=0-step=3500,1-1-0,44,1.000000,67.774272,NaN,-0.909091,-59.380211,NaN,w1l1,0
3557,im_sub-01,sub-01_epoch=0-step=500,1-1-0,44,1.000000,65.546218,NaN,-0.909091,-59.380211,NaN,w1l1,0
4137,im_sub-01,sub-01_epoch=0-step=5000,1-1-0,44,1.000000,69.729730,NaN,-0.909091,-59.380211,NaN,w1l1,0
2977,im_sub-01,sub-01_epoch=0-step=6500,1-1-0,44,0.090909,6.166008,NaN,-0.909091,-59.380211,NaN,w1l1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11715,sub-05,Late practice,8-3-9,0,NaN,NaN,NaN,NaN,NaN,NaN,w8l3,9
12295,sub-06,Early discovery,8-3-9,6,0.666667,69.068186,10.583333,0.133333,10.194895,-8.961299,w8l3,9
12585,sub-06,Early practice,8-3-9,6,0.833333,75.940503,3.608475,0.133333,10.194895,-8.961299,w8l3,9
12005,sub-06,Late discovery,8-3-9,6,0.833333,75.943115,5.003390,0.133333,10.194895,-8.961299,w8l3,9


In [62]:
def get_sub_type(row):
    if row['subject'] == 'ppo':
        return 'ppo'
    elif row['subject'].startswith('im'):
        return 'imi'
    else:
        return 'hum'

df['sub_type'] = df.apply(get_sub_type, axis=1)

df_select = (
    df
    .groupby(['scene_full_name', 'sub_type'])[
        ['delta_clr_tot', 'delta_spd_tot', 'MAD_mean']
    ]
    .mean()
    .reset_index()
)
df_select.head(20)

,scene_full_name,sub_type,delta_clr_tot,delta_spd_tot,MAD_mean
0,1-1-0,hum,0.000000,-0.400554,1.099498
1,1-1-0,imi,-0.767514,-39.831417,NaN
2,1-1-0,ppo,0.000000,14.225168,0.598918
3,1-1-1,hum,0.023333,5.380366,9.668016
4,1-1-1,imi,0.058900,-1.718801,NaN
5,1-1-1,ppo,0.341317,50.466879,18.252898
6,1-1-10,hum,0.000000,16.642659,5.661232
7,1-1-10,imi,0.029171,0.171553,NaN
8,1-1-10,ppo,-0.025000,18.316488,7.668340
9,1-1-11,hum,0.013333,27.685205,4.566335


In [78]:
thres_clr = 15
thres_spd = 15
thres_MAD = 15

In [79]:
#clearance contrast selections

ppo_d_im_u_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)

ppo_d_hum_u_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
imi_d_ppo_u_clearance = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
imi_d_hum_u_clearance = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
hum_d_ppo_u_clearance = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
hum_d_imi_u_clearance = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)

#clearance congruent selections
ppo_d_im_d_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
ppo_d_hum_d_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
imi_d_hum_d_clearance = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
ppo_u_im_u_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
ppo_u_hum_u_clearance = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)
imi_u_hum_u_clearance = (
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_clr, 'delta_clr_tot')['scene_full_name'])
)


In [80]:
print("Clearance Contrast Selections:")
print("PPO decrease & IM increase:", ppo_d_im_u_clearance)
print("PPO decrease & HUM increase:", ppo_d_hum_u_clearance)
print("IM decrease & PPO increase:", imi_d_ppo_u_clearance)
print("IM decrease & HUM increase:", imi_d_hum_u_clearance)
print("HUM decrease & PPO increase:", hum_d_ppo_u_clearance)
print("HUM decrease & IM increase:", hum_d_imi_u_clearance)

print("Clearance Congruent Selections:")
print("PPO decrease & IM decrease:", ppo_d_im_d_clearance)
print("PPO decrease & HUM decrease:", ppo_d_hum_d_clearance)
print("IM decrease & HUM decrease:", imi_d_hum_d_clearance)
print("PPO increase & IM increase:", ppo_u_im_u_clearance)
print("PPO increase & HUM increase:", ppo_u_hum_u_clearance)
print("IM increase & HUM increase:", imi_u_hum_u_clearance)

Clearance Contrast Selections:
PPO decrease & IM increase: {'1-3-9', '3-1-7'}
PPO decrease & HUM increase: {'8-2-15'}
IM decrease & PPO increase: {'8-3-0'}
IM decrease & HUM increase: {'8-2-13'}
HUM decrease & PPO increase: set()
HUM decrease & IM increase: {'1-2-10'}
Clearance Congruent Selections:
PPO decrease & IM decrease: {'5-1-0'}
PPO decrease & HUM decrease: set()
IM decrease & HUM decrease: set()
PPO increase & IM increase: {'4-1-7', '3-3-6'}
PPO increase & HUM increase: {'2-3-11'}
IM increase & HUM increase: set()


In [81]:
#speed contrast selections
ppo_d_im_u_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
ppo_d_hum_u_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
imi_d_ppo_u_speed = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
imi_d_hum_u_speed = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
hum_d_ppo_u_speed = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
hum_d_imi_u_speed = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)

# speed congruent selections
ppo_d_im_d_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
ppo_d_hum_d_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
imi_d_hum_d_speed = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
ppo_u_im_u_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
ppo_u_hum_u_speed = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)
imi_u_hum_u_speed = (
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_spd, 'delta_spd_tot')['scene_full_name'])
)   


In [82]:
print('Speed Contrast Selections:')
print("PPO decrease & IM increase:", ppo_d_im_u_speed)
print("PPO decrease & HUM increase:", ppo_d_hum_u_speed)
print("IM decrease & PPO increase:", imi_d_ppo_u_speed)
print("IM decrease & HUM increase:", imi_d_hum_u_speed)
print("HUM decrease & PPO increase:", hum_d_ppo_u_speed)    
print("HUM decrease & IM increase:", hum_d_imi_u_speed)

print('Speed Congruent Selections:')
print("PPO decrease & IM decrease:", ppo_d_im_d_speed)
print("PPO decrease & HUM decrease:", ppo_d_hum_d_speed)
print("IM decrease & HUM decrease:", imi_d_hum_d_speed)
print("PPO increase & IM increase:", ppo_u_im_u_speed)
print("PPO increase & HUM increase:", ppo_u_hum_u_speed)
print("IM increase & HUM increase:", imi_u_hum_u_speed)


Speed Contrast Selections:
PPO decrease & IM increase: set()
PPO decrease & HUM increase: set()
IM decrease & PPO increase: {'6-1-4'}
IM decrease & HUM increase: {'7-3-12'}
HUM decrease & PPO increase: {'6-2-0'}
HUM decrease & IM increase: {'1-2-10'}
Speed Congruent Selections:
PPO decrease & IM decrease: {'5-1-0'}
PPO decrease & HUM decrease: set()
IM decrease & HUM decrease: {'8-1-22', '6-2-1'}
PPO increase & IM increase: {'7-3-2'}
PPO increase & HUM increase: {'4-1-7'}
IM increase & HUM increase: {'1-3-9', '1-1-11'}


In [83]:
# mad contrast selections
'''ppo_d_im_u_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
ppo_d_hum_u_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)
'''imi_d_ppo_u_MAD = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)
imi_d_hum_u_MAD = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
hum_d_ppo_u_MAD = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)
'''hum_d_imi_u_MAD = (
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
# mad congruent selections
'''ppo_d_im_d_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
ppo_d_hum_d_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
)
'''imi_d_hum_d_MAD = (
    set(df_select[df_select['sub_type']=='imi'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nsmallest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
'''ppo_u_im_u_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''
ppo_u_hum_u_MAD = (
    set(df_select[df_select['sub_type']=='ppo'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)
'''imi_u_hum_u_MAD = (
    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
    &
    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])
)'''

"imi_u_hum_u_MAD = (\n    set(df_select[df_select['sub_type']=='imi'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])\n    &\n    set(df_select[df_select['sub_type']=='hum'].nlargest(thres_MAD, 'MAD_mean')['scene_full_name'])\n)"

In [84]:
print('MAD Contrast Selections:')
#print("PPO decrease & IM increase:", ppo_d_im_u_MAD)
print("PPO decrease & HUM increase:", ppo_d_hum_u_MAD)
#print("IM decrease & PPO increase:", imi_d_ppo_u_MAD)
#print("IM decrease & HUM increase:", imi_d_hum_u_MAD)
print("HUM decrease & PPO increase:", hum_d_ppo_u_MAD)
#print("HUM decrease & IM increase:", hum_d_imi_u_MAD)

print('MAD Congruent Selections:')
#print("PPO decrease & IM decrease:", ppo_d_im_d_MAD)
print("PPO decrease & HUM decrease:", ppo_d_hum_d_MAD)
#print("IM decrease & HUM decrease:", imi_d_hum_d_MAD)
#print("PPO increase & IM increase:", ppo_u_im_u_MAD)
print("PPO increase & HUM increase:", ppo_u_hum_u_MAD)
#print("IM increase & HUM increase:", imi_u_hum_u_MAD)


MAD Contrast Selections:
PPO decrease & HUM increase: set()
HUM decrease & PPO increase: set()
MAD Congruent Selections:
PPO decrease & HUM decrease: {'2-1-0', '1-1-0', '3-3-0', '6-1-0', '7-1-0', '3-1-0', '1-3-0', '4-1-0'}
PPO increase & HUM increase: {'5-2-2', '6-3-5', '6-3-3', '4-3-4', '6-3-4', '4-2-11', '4-3-8'}


In [91]:
# tendance in all metrics
print("Tendance in all metrics selections:")
print("PPO decrease & IM increase:", ppo_d_im_u_clearance | ppo_d_im_u_speed )
print("PPO decrease & HUM increase:", ppo_d_hum_u_clearance | ppo_d_hum_u_speed | ppo_d_hum_u_MAD)
print("IM decrease & PPO increase:", imi_d_ppo_u_clearance | imi_d_ppo_u_speed )
print("IM decrease & HUM increase:", imi_d_hum_u_clearance | imi_d_hum_u_speed )
print("HUM decrease & PPO increase:", hum_d_ppo_u_clearance | hum_d_ppo_u_speed | hum_d_ppo_u_MAD)
print("HUM decrease & IM increase:", hum_d_imi_u_clearance | hum_d_imi_u_speed )
#clearance congruent selections
print("PPO decrease & IM decrease:", ppo_d_im_d_speed | ppo_d_im_d_clearance)
print("PPO decrease & HUM decrease:", ppo_d_hum_d_MAD | ppo_d_hum_d_speed | ppo_d_hum_d_clearance)
print("IM decrease & HUM decrease:", imi_d_hum_d_speed | imi_d_hum_d_clearance)
print("PPO increase & IM increase:", ppo_u_im_u_speed | ppo_u_im_u_clearance)
print("PPO increase & HUM increase:", ppo_u_hum_u_MAD | ppo_u_hum_u_speed | ppo_u_hum_u_clearance)
print("IM increase & HUM increase:", imi_u_hum_u_speed | imi_u_hum_u_clearance)


Tendance in all metrics selections:
PPO decrease & IM increase: {'1-3-9', '3-1-7'}
PPO decrease & HUM increase: {'8-2-15'}
IM decrease & PPO increase: {'8-3-0', '6-1-4'}
IM decrease & HUM increase: {'7-3-12', '8-2-13'}
HUM decrease & PPO increase: {'6-2-0'}
HUM decrease & IM increase: {'1-2-10'}
PPO decrease & IM decrease: {'5-1-0'}
PPO decrease & HUM decrease: {'2-1-0', '1-1-0', '3-3-0', '6-1-0', '7-1-0', '3-1-0', '1-3-0', '4-1-0'}
IM decrease & HUM decrease: {'8-1-22', '6-2-1'}
PPO increase & IM increase: {'7-3-2', '4-1-7', '3-3-6'}
PPO increase & HUM increase: {'5-2-2', '6-3-5', '6-3-3', '4-1-7', '4-3-4', '6-3-4', '2-3-11', '4-2-11', '4-3-8'}
IM increase & HUM increase: {'1-3-9', '1-1-11'}


Tendance in all metrics selections:
PPO decrease & IM increase: {'1-3-9', '3-1-7 ok'}
PPO decrease & HUM increase: {'8-2-15 no'}
IM decrease & PPO increase: {'8-3-0ok', '6-1-4 b'}
IM decrease & HUM increase: {'7-3-12 no', '8-2-13 '}
HUM decrease & PPO increase: {'6-2-0'}
HUM decrease & IM increase: {'1-2-10'}
PPO decrease & IM decrease: {'5-1-0'}
PPO decrease & HUM decrease: {'2-1-0', '1-1-0', '3-3-0', '6-1-0', '7-1-0', '3-1-0', '1-3-0', '4-1-0'}
IM decrease & HUM decrease: {'8-1-22', '6-2-1'}
PPO increase & IM increase: {'7-3-2', '4-1-7', '3-3-6'}
PPO increase & HUM increase: {'5-2-2', '6-3-5', '6-3-3', '4-1-7', '4-3-4', '6-3-4', '2-3-11', '4-2-11', '4-3-8'}
IM increase & HUM increase: {'1-3-9', '1-1-11'}

Selected :
Tendance in all metrics selections (delta MAD):
PPO decrease & IM increase: {'3-1-7', '1-1-2', '2-1-5'} 
PPO decrease & HUM increase: {'6-2-14'}
IM decrease & PPO increase: {'4-1-11', '3-2-13'}
IM decrease & HUM increase: {'1-3-7', '2-3-9'}
HUM decrease & PPO increase: {'8-1-1', '8-1-4'}
HUM decrease & IM increase: {'1-2-1', '3-2-1', '1-1-14', '2-1-5'}
PPO decrease & IM decrease: {'1-2-4', '2-1-8', '2-1-4'}
PPO decrease & HUM decrease: {'2-1-5', '4-2-4'}
IM decrease & HUM decrease: {'1-1-0', '3-2-2', '1-1-5'}
PPO increase & IM increase: {'2-3-10', '4-2-13', '3-2-14', '4-1-6', '2-3-11'}
PPO increase & HUM increase: {'4-1-6', '8-1-24', '6-2-4'}
IM increase & HUM increase: {'1-3-9', '4-1-6', '1-1-7', '2-1-2'}